<a href="https://colab.research.google.com/github/Mmengyw/Capstone-Updated/blob/main/Depth%20Prediction/Optical_Flow_4_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importation**

In [1]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow
import os
#os.chdir("/Users/dagu/Downloads/Zendar/Code Depth Improvement")

In [2]:
!git clone https://github.com/Mmengyw/Capstone-Updated.git
!ls
os.chdir("Capstone-Updated/Videos")

Cloning into 'Capstone-Updated'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 138 (delta 64), reused 14 (delta 5), pack-reused 18
Receiving objects: 100% (138/138), 105.87 MiB | 14.60 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Capstone-Updated  sample_data


**calcOpticalFlowPyrLK() function**

Arguments
*   old_gray : old frame array
* gray_frame : current frame
* old_point : array with old point
* None : empy param
* **lk_params : some configuration about the function



In [4]:
cap = cv2.VideoCapture(0)

import numpy as np
import cv2 as cv
cap = cv.VideoCapture(cv.samples.findFile("test12.mp4"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    #cv2_imshow(bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
print(bgr)
print(hsv)

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: ignored

In [ ]:
# The video feed is read in as a VideoCapture object
cap = cv.VideoCapture("test12.mp4")
# ret = a boolean return value from getting the frame, first_frame = the first frame in the entire video sequence
ret, first_frame = cap.read()
# Converts frame to grayscale because we only need the luminance channel for detecting edges - less computationally expensive
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
# Creates an image filled with zero intensities with the same dimensions as the frame
mask = np.zeros_like(first_frame)
# Sets image saturation to maximum
mask[..., 1] = 255
#print(mask)
#intializing the flow
gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

#dimensions of the frame and number of frames
nb_row, nb_col, prout = np.shape(first_frame)
frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
print(nb_row, nb_col,frame_count)
#Array of the movement of each pixels for each frame across time
#Mov_pix_optflow = np.zeros((frame_count, nb_row, nb_col, 2))

while(cap.isOpened()):
    # ret = a boolean return value from getting the frame, frame = the current frame being projected in the video
    ret, frame = cap.read()

    # Opens a new wqqindow and displays the input frame
    #cv2_imshow(frame)

    # Converts each frame to grayscale - we previously only converted the first frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculates dense optical flow by Farneback method
    # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
    flow = cv.calcOpticalFlowFarneback(prev=prev_gray,
                                      next=gray, flow=flow,
                                      pyr_scale=0.5, levels=3, winsize=15,
                                      iterations=3, poly_n=5, poly_sigma=1.2,
                                      flags=0)
    #flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    #print(flow)
    # Computes the magnitude and angle of the 2D vectors
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])

    # Computes the dx and dy of the 2D vectors
    dx=magnitude*np.cos(angle)
    dy=magnitude*np.sin(angle)
    #print('dx=',dx)
    #print('dy=',dy)

    Mov_pix_optflow.append()


    # Sets image hue according to the optical flow direction
    #mask[..., 0] = angle * 180 / np.pi / 2

    # Sets image value according to the optical flow magnitude (normalized)
    #mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    print('STOP')

    # Converts HSV to RGB (BGR) color representation
    #rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

    # Opens a new window and displays the output frame
    #cv2_imshow(rgb)

    # Updates previous frame
    prev_gray = gray
    # Frames are read by intervals of 1 millisecond. The programs breaks out of the while loop when the user presses the 'q' key
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
# The following frees up resources and closes all windows
cap.release()
cv.destroyAllWindows()

'''
dx= [[ 1.66647096e-09 -8.85083007e-10  2.69949951e-09 ...  9.02536690e-01
   6.76709056e-01  4.62470412e-01]
 ...
 [ 3.59032420e-04  9.28807654e-04  1.61183486e-03 ...  7.55291879e-02
   5.46804480e-02  5.34909889e-02]]


dy= [[ 4.8883432e-11  2.3772040e-10  5.1093835e-10 ...  1.9181054e+00
   1.4482435e+00  9.5387548e-01]
 ...
 [-1.3773385e-04 -1.4980065e-04 -1.4452441e-04 ...  1.2933868e-01
   8.1860811e-02  3.5035118e-02]]

max = order of 10 pixels
min = 0
   '''

1240 1616 3910


In [ ]:

#coordinates of a pixel (u,v), dimension=pixel
#coordinates of a reel point in space (x,y,z), dimension=meter


#working on frame_k and frame_k1

##Optical flow processing
def convert_Optfl(Optfl_map):
    du=magni*np.cos(angle)
    dv=magni*np.cos(angle)
    return #array= [[du,dv]] for each pixel
Optfl_map=#optical flow (magnitude and angle) (arg=frame_k, frame_k1)
Optfl_map=convert_Optfl(Optfl_map)#optical flow map in terms of du, dv for each pixel 

depth_map_k=#put the depth_map from midas (z)
Pos_map_k=#pinhole map with the depth map k: (x,y,z) # position (x,y,z) for each pixels of frame k




depth_map_k1=#put the depth_map from midas (z) for the next frame
interpol_depth_map_k1= #f(depth_map_k1)











def get_next_pos_midas(Pos_map_k,interpol_depth_map_k1,Optfl_map):
    #need 2)
    pos_next_frame_midas=np.zeros(N,M)  #position in meters of the next frame
    Optfl_depth_map_k1=np.zeros(N,M)    # each pixel value is the next depth of the pixel
    deplacement=np.zeros(N,M,3)        #dx,dy,dz in meters  

    for v in range(0,N):
        for u in range(0,M):   
            du,dv = Optfl_map [v][u]
            Optfl_depth_map_k1 [v][u] = interpol_depth_map_k1 [v+dv][u+du] # each pixel value is the next depth of the pixel
            z1 = Optfl_depth_map_k1 [v][u]  #ref k+1

            x,y,z = Pos_map_k [v][u]    #ref k
            xrefk1,yrefk1,zrefk1 = #utiliser la matrice de tranformation de la voiture
            x1,y1 = #fonction pinhole([v+dv][u+du]) #ref k1

            deplacement [v][u] = [x1-xrefk1, y1-yrefk1, z1-zrefk1]
            pos_next_frame_midas [v][u] = [x1,y1,z1]

    return (pos_next_frame_midas,deplacement)#deplacement dx.dy,dz in meters for each pixel

Pos_map_k1_midas, deplacement_k=get_next_pos_midas(Pos_map_k,interpol_depth_map_k1,Optfl_map)